In [1]:
import geemap
import pandas as pd
import numpy as np
import pdb
from IPython.display import display
import ee
import os

In [2]:
import ee
import geemap
service_account = ' boba-account@boba-430314.iam.gserviceaccount.com '
credentials = ee.ServiceAccountCredentials(service_account,'C:/Users/arj26323/OneDrive - University of Georgia/Documents/GEE AUTH/boba-430314-a19be859bfca.json')

ee.Initialize(credentials)

print(geemap.__version__)

#ee.Initialize()

ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

In [3]:
# Define the region of interest for Georgia and Iowa
iowa = geemap.shp_to_ee('F:/US states/iowa.shp')
georgia = geemap.shp_to_ee('F:/US states/georgia.shp')

In [4]:
dem = ee.ImageCollection('USGS/3DEP/1m') \
    .filterBounds(iowa) \
    .mosaic()

dem_ga = ee.ImageCollection('USGS/3DEP/1m') \
    .filterBounds(georgia) \
    .mosaic()

In [7]:
#Add date to image
def addDate(image):
    img_date = ee.Date(image.date())
    img_date = ee.Number.parse(img_date.format('YYYYMMdd'))
    return image.addBands(ee.Image(img_date).rename('imagedate').toInt())

def addNDVI(image):
    ndvi = ee.Image(0).expression(
        '((NIR-RED)/(NIR+RED))', {
            'NIR': image.select('N'),
            'RED': image.select('R')
        })
    
    return image.addBands(ndvi.rename('ndvi'))

def addNDWI(image):
    ndwi = ee.Image(0).expression(
        '((GREEN-NIR)/(GREEN+NIR))', {
            'NIR': image.select('N'),
            'GREEN': image.select('G')
        })
    
    return image.addBands(ndwi.rename('ndwi'))

iowa_mosaic_2010 = ee.ImageCollection('USDA/NAIP/DOQQ') \
    .filterBounds(iowa) \
    .filterDate('2010-01-01', '2010-12-31') \
    .map(addDate) \
    .mosaic() \
    .addBands(dem.select('elevation'))

iowa_naip = ee.ImageCollection('USDA/NAIP/DOQQ') \
    .filterBounds(iowa) \
    .filterDate('2010-01-01', '2010-12-31') 

georgia_naip = ee.ImageCollection('USDA/NAIP/DOQQ') \
    .filterBounds(georgia) \
    .filterDate('2017-01-01', '2018-12-31')

ga_im = addNDWI(addNDVI(ee.Image('USDA/NAIP/DOQQ/m_3108101_se_17_060_20220130'))).addBands(dem_ga.select('elevation'))
ia_im = addNDWI(addNDVI(ee.Image('USDA/NAIP/DOQQ/m_4109451_nw_15_1_20100811'))).addBands(dem.select('elevation'))

In [8]:
ia_im

In [11]:
Map = geemap.Map(center=[32, -82], zoom=9)

Map.addLayer(ga_im, {}, 'GA NAIP')

# Map

In [8]:
out_dir = os.path.expanduser('~F:/Iowa BMP/Geotiffs/GA/Tests')
out_tif = os.path.join(out_dir, 'ga_im.tif')

image = ga_im
aoi = image.geometry()

# geemap.ee_export_image(
#     image, filename=out_tif, scale=1, region=aoi, file_per_band=False
# )


In [14]:
#IOWA

aoi = geemap.shp_to_ee('F:/Iowa BMP/Iowa clipped shapefiles/aoi/roi.shp').geometry() #use aoi5/aoi6 for addedbands, aoi3 else
image = ia_im

out_dir = os.path.expanduser('~F:/Iowa BMP/Geotiffs/IA/Tests')
out_tif = os.path.join(out_dir, 'ia_im.tif')

image = image.clip(aoi).unmask()

# geemap.ee_export_image(
#     image, filename=out_tif, scale=1, region=aoi, file_per_band=False
# )

Generating URL ...
Please wait ...
Data downloaded to F:\Iowa BMP\Geotiffs\IA\Tests\ia_im.tif


In [12]:
ia_im

In [11]:
print("feesh")